In [1]:
!pip install transformers datasets evaluate accelerate optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate
import optuna

In [3]:
dataset = load_dataset("emotion")
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
print(tokenized_datasets["train"][0])


{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [6]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import transformers
print(transformers.__version__)


4.57.1


In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./emotion_model",
    eval_strategy="epoch",   # ✅ correct keyword
    save_strategy="epoch",         # save model after every epoch
    save_total_limit=2,            # optional: keep only last 2 checkpoints
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_opt",
    logging_strategy="epoch"       # ✅ optional: logs training loss each epoch
)


In [9]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}


In [10]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=6
    )


In [11]:
trainer = Trainer(
    model_init=model_init,   # ✅ not model=model
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-2239306170.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 4),
    }


In [13]:
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    n_trials=3,   # runs 3 experiments automatically
)


[I 2025-10-23 15:12:16,836] A new study created in memory with name: no-name-8f39166c-8444-4e9d-866f-1c616ecc1d4f
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phoenix17081708 (phoenix17081708-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.393600,0.186726,0.930000,0.930572
2,0.136700,0.158536,0.935500,0.934996


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.393600,0.186726,0.930000,0.930572
2,0.136700,0.158536,0.935500,0.934996
3,0.095200,0.156102,0.943000,0.942965
4,0.063900,0.192996,0.937000,0.937036


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.422400,0.224871,0.928000,0.928692
2,0.155400,0.176376,0.944500,0.944342


[I 2025-10-23 15:32:12,486] Trial 1 finished with value: 0.9443422616505562 and parameters: {'learning_rate': 2.020849846823505e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 2}. Best is trial 1 with value: 0.9443422616505562.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁███
train/global_step,▁▁███
train/grad_norm,▁█
train/learning_rate,█▁
+1,...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.560000,0.199761,0.923500,0.924276
2,0.156400,0.146147,0.937000,0.937130
3,0.104500,0.135621,0.941000,0.940958


[I 2025-10-23 15:41:04,284] Trial 2 finished with value: 0.9409577885194419 and parameters: {'learning_rate': 2.320122623229832e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 3}. Best is trial 1 with value: 0.9443422616505562.


In [14]:
print("🏆 Best Run Summary:")
print(best_run)

# ✅ Apply the best hyperparameters to your trainer
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

# ✅ Train the final model with the best hyperparameters
train_output = trainer.train()

# ✅ Save the fine-tuned model
trainer.save_model("./final_emotion_model")
print("✅ Final fine-tuned model saved to './final_emotion_model'")


🏆 Best Run Summary:
BestRun(run_id='1', objective=0.9443422616505562, hyperparameters={'learning_rate': 2.020849846823505e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 2}, run_summary=None)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.422400,0.224871,0.928000,0.928692
2,0.155400,0.176376,0.944500,0.944342


✅ Final fine-tuned model saved to './final_emotion_model'


In [15]:
# ✅ Save the trained model and tokenizer manually
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/final_emotion_model"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Model and tokenizer saved to:", save_path)


Mounted at /content/drive
✅ Model and tokenizer saved to: /content/drive/MyDrive/final_emotion_model


In [16]:
results = trainer.evaluate(tokenized_datasets["test"])
print(results)


{'eval_loss': 0.20792682468891144, 'eval_accuracy': 0.928, 'eval_f1': 0.927807805715406, 'eval_runtime': 7.7831, 'eval_samples_per_second': 256.967, 'eval_steps_per_second': 16.06, 'epoch': 2.0}


In [17]:
from transformers import pipeline
baseline_model = pipeline(
    "text-classification",
    model="distilbert-base-uncased",
    tokenizer="distilbert-base-uncased"
)
print(baseline_model("I am so happy and grateful today!"))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.5223959684371948}]


In [18]:
pipe = pipeline(
    "text-classification",
    model="./final_emotion_model",
    tokenizer="./final_emotion_model"
)


Device set to use cuda:0


In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_path = "./final_emotion_model"  # your saved final model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

print(pipe("I’m so excited for the weekend!"))


Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9990422129631042}]


In [20]:
# Map numeric labels to text labels
id2label = dataset["train"].features["label"].int2str

# Example sentence
text = "I’m so excited for the weekend!"
result = pipe(text)[0]

# Decode numeric label if necessary
predicted_label = result["label"]
if predicted_label.startswith("LABEL_"):
    # Extract the number and map it back
    label_id = int(predicted_label.split("_")[1])
    predicted_label = id2label(label_id)

print(f"Text: {text}")
print(f"Predicted emotion: {predicted_label} ({result['score']:.2f})")


Text: I’m so excited for the weekend!
Predicted emotion: joy (1.00)


In [21]:
model.save_pretrained("./final_emotion_model")
tokenizer.save_pretrained("./final_emotion_model")


('./final_emotion_model/tokenizer_config.json',
 './final_emotion_model/special_tokens_map.json',
 './final_emotion_model/vocab.txt',
 './final_emotion_model/added_tokens.json',
 './final_emotion_model/tokenizer.json')

In [22]:
#Model Evaluation

import evaluate
import numpy as np

# Baseline (pre-fine-tuned) model
baseline_pipe = pipeline("text-classification", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")

texts = dataset["test"]["text"][:200]   # evaluate on a small subset for speed
true_labels = dataset["test"]["label"][:200]

# Get baseline predictions
baseline_preds = [baseline_pipe(t)[0]['label'] for t in texts]

# Map predicted LABEL_X → numeric
baseline_pred_ids = [int(p.split("_")[1]) if "LABEL_" in p else 0 for p in baseline_preds]

# Compute baseline metrics
from sklearn.metrics import accuracy_score, f1_score
baseline_acc = accuracy_score(true_labels, baseline_pred_ids)
baseline_f1 = f1_score(true_labels, baseline_pred_ids, average="weighted")

print(f"📊 Baseline (Pre-Fine-Tuned) Model:")
print(f"Accuracy: {baseline_acc*100:.2f}%")
print(f"F1 Score: {baseline_f1*100:.2f}%")

# Compare with your fine-tuned results
print("\n📈 Fine-Tuned Model:")
print(f"Accuracy: {results['eval_accuracy']*100:.2f}%")
print(f"F1 Score: {results['eval_f1']*100:.2f}%")

improvement_acc = (results['eval_accuracy'] - baseline_acc) * 100
improvement_f1 = (results['eval_f1'] - baseline_f1) * 100

print(f"\n🚀 Improvement after Fine-Tuning: +{improvement_acc:.2f}% accuracy, +{improvement_f1:.2f}% F1 score.")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


📊 Baseline (Pre-Fine-Tuned) Model:
Accuracy: 31.00%
F1 Score: 14.67%

📈 Fine-Tuned Model:
Accuracy: 92.80%
F1 Score: 92.78%

🚀 Improvement after Fine-Tuning: +61.80% accuracy, +78.11% F1 score.


In [23]:
#Error Analysis
import pandas as pd
import numpy as np

# Get predictions from your trainer
predictions = trainer.predict(tokenized_datasets["test"])
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

# Convert numeric labels → emotion names in one go
id2label = dataset["train"].features["label"].int2str
true_labels = id2label(y_true)       # ✅ convert entire array
predicted_labels = id2label(y_pred)  # ✅ convert entire array

# Get original test texts
texts = dataset["test"]["text"]

# Create DataFrame for analysis
df_errors = pd.DataFrame({
    "text": texts,
    "true_label": true_labels,
    "predicted_label": predicted_labels
})

# Filter misclassified examples
df_misclassified = df_errors[df_errors["true_label"] != df_errors["predicted_label"]]

# Display 5 random misclassified samples
print("⚠️ Sample Misclassified Examples:\n")
print(df_misclassified.sample(5, random_state=42)[["text", "true_label", "predicted_label"]])


⚠️ Sample Misclassified Examples:

                                                   text true_label  \
1729  i felt like id developed feelings for this guy...        joy   
314   i will nolonger tell anybody how i feel or wha...    sadness   
1185  i start to feel a little overwhelmed knowing i...   surprise   
1383  this monday i took a math bs test and flunked ...    sadness   
795   i feel a strange gratitude for the hated israe...   surprise   

     predicted_label  
1729            love  
314            anger  
1185            fear  
1383             joy  
795             fear  


In [25]:
#Interface

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# ✅ Load local fine-tuned model
model_path = "./final_emotion_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Create pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

print("🎭 Emotion Detection Interface")
print("Type a sentence (or 'quit' to exit):\n")

while True:
    text = input("You: ")   # 👈 THIS is where you give input manually
    if text.lower() in ["quit", "exit"]:
        print("Session ended.")
        break

    result = pipe(text)[0]
    label = result["label"]
    if label.startswith("LABEL_"):
        label_id = int(label.split("_")[1])
        label = dataset["train"].features["label"].int2str(label_id)

    confidence = round(result["score"] * 100, 2)
    print(f"🤖 Predicted Emotion: {label} ({confidence}% confidence)\n")


Device set to use cuda:0


🎭 Emotion Detection Interface
Type a sentence (or 'quit' to exit):

You: quit
Session ended.


In [26]:
sentences = [
    "I’m so proud of my best friend!",
    "I can’t stop crying today.",
    "That movie scared me a lot.",
    "I feel peaceful and grateful.",
    "I hate how they ignored me!"
]

for text in sentences:
    result = pipe(text)[0]
    if result["label"].startswith("LABEL_"):
        label_id = int(result["label"].split("_")[1])
        label_name = id2label(label_id)
    else:
        label_name = result["label"]
    print(f"{text} → {label_name} ({result['score']:.2f})")


I’m so proud of my best friend! → joy (1.00)
I can’t stop crying today. → sadness (0.99)
That movie scared me a lot. → fear (1.00)
I feel peaceful and grateful. → joy (1.00)
I hate how they ignored me! → sadness (0.97)
